Part A2.1

In [12]:
from pynq import Overlay
from pynq.overlays.base import BaseOverlay
import threading
import time
from enum import Enum

# Define the possible states for each philosopher
class PhilosopherState(Enum):
    STARVING = 0    # Philosopher is waiting for forks
    EATING = 1      # Philosopher has both forks and is eating
    NAPPING = 2     # Philosopher is done eating and is napping

class DiningPhilosophers:
    def __init__(self):
        # System configuration constants
        self.NUM_PHILOSOPHERS = 5    # Total number of philosophers
        self.EATING_TIME = 2         # Time (in seconds) a philosopher spends eating
        self.NAPPING_TIME = 3        # Time (in seconds) a philosopher spends napping
        
        # LED blinking intervals for different states
        self.EATING_BLINK_INTERVAL = 0.1  # Fast blink rate when eating (100ms)
        self.NAPPING_BLINK_INTERVAL = 0.5  # Slow blink rate when napping (500ms)

        # Initialize PYNQ hardware
        self.base = BaseOverlay("base.bit")        # Load the base overlay
        self.leds = [self.base.leds[i] for i in range(4)]      # Get all 4 LEDs
        self.buttons = [self.base.buttons[i] for i in range(4)] # Get all 4 buttons
        
        # Create mutex locks for forks (one lock per fork)
        self.forks = [threading.Lock() for _ in range(self.NUM_PHILOSOPHERS)]
        
        # Initialize state tracking
        self.philosopher_states = [PhilosopherState.STARVING] * self.NUM_PHILOSOPHERS
        self.philosophers = []    # List to keep track of philosopher threads
        self.running = True      # Global flag to control program execution
        
        # Initial LED setup - all off
        for led in self.leds:
            led.off()

    def check_buttons(self):
        """
        Continuously monitor buttons for program termination.
        Any button press will stop the simulation.
        """
        while self.running:
            # Check if any button is pressed
            if any(button.read() for button in self.buttons):
                print("\nButton pressed - Stopping the simulation...")
                self.running = False
                break
            time.sleep(0.1)

    def update_led(self, philosopher_id):
        """
        Control LED behavior based on philosopher's state.
        Each state has a different LED pattern:
        - Eating: Fast blinking
        - Napping: Slow blinking
        - Starving: LED off
        
        Args:
            philosopher_id (int): The ID of the philosopher (0-4)
        """
        while self.running:
            try:
                state = self.philosopher_states[philosopher_id]
                
                # For philosopher 4, we reuse LED 3 since we only have 4 LEDs
                led = self.leds[philosopher_id if philosopher_id < 4 else 3]
                
                if state == PhilosopherState.EATING:
                    # Fast blinking for eating state
                    led.toggle()
                    time.sleep(self.EATING_BLINK_INTERVAL)
                
                elif state == PhilosopherState.NAPPING:
                    # Slow blinking for napping state
                    led.toggle()
                    time.sleep(self.NAPPING_BLINK_INTERVAL)
                
                else:  # STARVING
                    # LED off when starving
                    led.off()
                    time.sleep(0.1)
                
            except Exception as e:
                if self.running:
                    print(f"Error updating LED {philosopher_id}: {e}")
                time.sleep(0.1)

    def philosopher_action(self, philosopher_id):
        """
        Implements the main behavior loop for each philosopher.
        Philosophers cycle through three states:
        1. Starving (waiting for forks)
        2. Eating (when they have both forks)
        3. Napping (after eating)
        
        Args:
            philosopher_id (int): The ID of the philosopher (0-4)
        """
        # Determine left and right fork IDs
        left_fork = philosopher_id
        right_fork = (philosopher_id + 1) % self.NUM_PHILOSOPHERS
        
        while self.running:
            # Set initial state to starving
            self.philosopher_states[philosopher_id] = PhilosopherState.STARVING
            
            # Implement deadlock prevention by having odd philosophers
            # pick up forks in opposite order from even philosophers
            if philosopher_id % 2 == 0:
                forks = [self.forks[left_fork], self.forks[right_fork]]
            else:
                forks = [self.forks[right_fork], self.forks[left_fork]]
            
            # Try to acquire both forks
            with forks[0]:
                with forks[1]:
                    # Successfully got both forks - start eating
                    self.philosopher_states[philosopher_id] = PhilosopherState.EATING
                    time.sleep(self.EATING_TIME)
            
            # Done eating, release forks and take a nap
            self.philosopher_states[philosopher_id] = PhilosopherState.NAPPING
            time.sleep(self.NAPPING_TIME)

    def start(self):
        """
        Initialize and start all necessary threads:
        - Button monitoring thread
        - LED control threads
        - Philosopher behavior threads
        """
        # Start button monitoring
        button_thread = threading.Thread(target=self.check_buttons)
        button_thread.daemon = True
        button_thread.start()

        # Start LED control threads
        led_threads = []
        for i in range(self.NUM_PHILOSOPHERS):
            thread = threading.Thread(target=self.update_led, args=(i,))
            thread.daemon = True
            led_threads.append(thread)
            thread.start()

        # Start philosopher threads
        for i in range(self.NUM_PHILOSOPHERS):
            thread = threading.Thread(target=self.philosopher_action, args=(i,))
            thread.daemon = True
            self.philosophers.append(thread)
            thread.start()

    def stop(self):
        """Clean up and shut down the simulation"""
        self.running = False
        time.sleep(0.2)  # Give threads time to complete
        # Turn off all LEDs
        for led in self.leds:
            led.off()

def main():
    """
    Main program entry point.
    Creates and runs the dining philosophers simulation.
    Simulation runs until a button is pressed.
    """
    dining = DiningPhilosophers()
    dining.start()
    
    # Keep the main thread running until button press
    while dining.running:
        time.sleep(0.1)
    
    # Clean up when button is pressed
    dining.stop()

if __name__ == "__main__":
    main()


Button pressed - Stopping the simulation...


Part A2.2

In [13]:
from pynq import Overlay
from pynq.overlays.base import BaseOverlay
import threading
import time
import random
from enum import Enum

class PhilosopherState(Enum):
    STARVING = 0    # Philosopher is waiting for forks
    EATING = 1      # Philosopher has both forks and is eating
    NAPPING = 2     # Philosopher is done eating and is napping

class DiningPhilosophers:
    def __init__(self):
        # System configuration constants
        self.NUM_PHILOSOPHERS = 5    # Total number of philosophers
        
        # Time boundaries (in seconds)
        self.MIN_EATING_TIME = 2     # Minimum time to eat
        self.MAX_EATING_TIME = 4     # Maximum time to eat
        self.MIN_NAPPING_TIME = 1    # Minimum time to nap
        self.MAX_NAPPING_TIME = 2    # Maximum time to nap (less than MIN_EATING_TIME)
        
        # LED blinking intervals for different states
        self.EATING_BLINK_INTERVAL = 0.1  # Fast blink rate when eating (100ms)
        self.NAPPING_BLINK_INTERVAL = 0.5  # Slow blink rate when napping (500ms)

        # Initialize PYNQ hardware
        self.base = BaseOverlay("base.bit")        # Load the base overlay
        self.leds = [self.base.leds[i] for i in range(4)]      # Get all 4 LEDs
        self.buttons = [self.base.buttons[i] for i in range(4)] # Get all 4 buttons
        
        # Create mutex locks for forks (one lock per fork)
        self.forks = [threading.Lock() for _ in range(self.NUM_PHILOSOPHERS)]
        
        # Initialize state tracking
        self.philosopher_states = [PhilosopherState.STARVING] * self.NUM_PHILOSOPHERS
        self.philosophers = []    # List to keep track of philosopher threads
        self.running = True      # Global flag to control program execution
        
        # Initial LED setup - all off
        for led in self.leds:
            led.off()

    def get_random_time(self, is_eating):
        """
        Generate random time duration for eating or napping.
        Ensures napping time is always less than minimum eating time
        to prevent starvation.
        
        Args:
            is_eating (bool): True if generating eating time, False for napping time
            
        Returns:
            float: Random time duration in seconds
        """
        if is_eating:
            return random.uniform(self.MIN_EATING_TIME, self.MAX_EATING_TIME)
        else:
            return random.uniform(self.MIN_NAPPING_TIME, self.MAX_NAPPING_TIME)

    def check_buttons(self):
        """
        Continuously monitor buttons for program termination.
        Any button press will stop the simulation.
        """
        while self.running:
            if any(button.read() for button in self.buttons):
                print("\nButton pressed - Stopping the simulation...")
                self.running = False
                break
            time.sleep(0.1)

    def update_led(self, philosopher_id):
        """
        Control LED behavior based on philosopher's state.
        Each state has a different LED pattern:
        - Eating: Fast blinking
        - Napping: Slow blinking
        - Starving: LED off
        
        Args:
            philosopher_id (int): The ID of the philosopher (0-4)
        """
        while self.running:
            try:
                state = self.philosopher_states[philosopher_id]
                led = self.leds[philosopher_id if philosopher_id < 4 else 3]
                
                if state == PhilosopherState.EATING:
                    led.toggle()
                    time.sleep(self.EATING_BLINK_INTERVAL)
                elif state == PhilosopherState.NAPPING:
                    led.toggle()
                    time.sleep(self.NAPPING_BLINK_INTERVAL)
                else:  # STARVING
                    led.off()
                    time.sleep(0.1)
                
            except Exception as e:
                if self.running:
                    print(f"Error updating LED {philosopher_id}: {e}")
                time.sleep(0.1)

    def philosopher_action(self, philosopher_id):
        """
        Implements the main behavior loop for each philosopher.
        Uses random times for eating and napping while ensuring
        napping time is always shorter than eating time.
        
        Args:
            philosopher_id (int): The ID of the philosopher (0-4)
        """
        left_fork = philosopher_id
        right_fork = (philosopher_id + 1) % self.NUM_PHILOSOPHERS
        
        while self.running:
            self.philosopher_states[philosopher_id] = PhilosopherState.STARVING
            
            # Deadlock prevention with asymmetric fork pickup
            if philosopher_id % 2 == 0:
                forks = [self.forks[left_fork], self.forks[right_fork]]
            else:
                forks = [self.forks[right_fork], self.forks[left_fork]]
            
            # Try to acquire both forks
            with forks[0]:
                with forks[1]:
                    # Successfully got both forks - start eating
                    self.philosopher_states[philosopher_id] = PhilosopherState.EATING
                    eating_time = self.get_random_time(is_eating=True)
                    time.sleep(eating_time)
            
            # Done eating, release forks and take a nap
            self.philosopher_states[philosopher_id] = PhilosopherState.NAPPING
            napping_time = self.get_random_time(is_eating=False)
            time.sleep(napping_time)

    def start(self):
        """
        Initialize and start all necessary threads:
        - Button monitoring thread
        - LED control threads
        - Philosopher behavior threads
        """
        # Start button monitoring
        button_thread = threading.Thread(target=self.check_buttons)
        button_thread.daemon = True
        button_thread.start()

        # Start LED control threads
        led_threads = []
        for i in range(self.NUM_PHILOSOPHERS):
            thread = threading.Thread(target=self.update_led, args=(i,))
            thread.daemon = True
            led_threads.append(thread)
            thread.start()

        # Start philosopher threads
        for i in range(self.NUM_PHILOSOPHERS):
            thread = threading.Thread(target=self.philosopher_action, args=(i,))
            thread.daemon = True
            self.philosophers.append(thread)
            thread.start()

    def stop(self):
        """Clean up and shut down the simulation"""
        self.running = False
        time.sleep(0.2)  # Give threads time to complete
        for led in self.leds:
            led.off()

def main():
    """
    Main program entry point.
    Creates and runs the dining philosophers simulation with random timing.
    Simulation runs until a button is pressed.
    """
    dining = DiningPhilosophers()
    dining.start()
    
    # Keep the main thread running until button press
    while dining.running:
        time.sleep(0.1)
    
    # Clean up when button is pressed
    dining.stop()

if __name__ == "__main__":
    main()


Button pressed - Stopping the simulation...
